In [1]:
import gurobipy as gp
from gurobipy import GRB
from eventlet.timeout import Timeout

# import auxillary packages
import requests  # for loading the example source code
import openai
import json

# import flaml and autogen
from flaml import autogen
from flaml.autogen.agentchat import Agent, UserProxyAgent
from EnergySaverLLM.Agent import ChargingAgent, run_with_exec

In [2]:
# global params_filepath
params_filepath = "EnergySaverLLM/Model/params/EVCharging.json"

In [3]:
config_list = autogen.config_list_from_json(
    env_or_file = "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-3.5-turbo"],
    },
)

In [4]:
autogen.oai.ChatCompletion.start_logging()

In [5]:
code_path = "EnergySaverLLM/Model/EVCharging.py"

In [6]:
with open(code_path) as f:
    code = f.read()

In [7]:
run_with_exec(code)

{'0': 0.0003, '1': 0.0003, '2': 0.0003, '3': 0.0003, '4': 0.0003, '5': 0.0003, '6': 0.0003, '7': 0.00035, '8': 0.00035, '9': 0.00035, '10': 0.00035, '11': 0.0003, '12': 0.0003, '13': 0.0003, '14': 0.0003, '15': 0.0003, '16': 0.0003, '17': 0.0003, '18': 0.0003, '19': 0.00035, '20': 0.00035, '21': 0.00035, '22': 0.0003, '23': 0.0003}
Restricted license - for non-production use only - expires 2024-10-28
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 48 columns and 96 nonzeros
Model fingerprint: 0x7e2fc113
Variable types: 0 continuous, 48 integer (0 binary)
Coefficient statistics:
  Matrix range     [3e-04, 1e+00]
  Objective range  [8e-03, 8e-03]
  Bounds range     [1e+04, 1e+04]
  RHS range        [2e+01, 5e+04]
Presolve removed 2 rows and 48 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iteration

'Optimization problem solved. The objective value is: 382.19999999999993'

In [13]:
globals()['params_filepath']

'EnergySaverLLM/Model/params/EVCharging.json'

In [10]:
run_with_exec(code)

NameError("name 'params_filepath' is not defined")

In [7]:
# show the first head and tail of the source code
print("\n".join(code.split("\n")[:10]))
print(".\n" * 3)
print("\n".join(code.split("\n")[-10:]))

from datetime import datetime
from gurobipy import GRB, Model
import json

# Fetching data from JSON file
stored_params = json.load(open(params_filepath))

start_charge_level = stored_params['start_charge']
end_charge_level = stored_params['default_end_charge']

.
.
.

    values = model.getAttr("X", all_vars)
    names = model.getAttr("VarName", all_vars)

    for i, time_str in enumerate(keys):
        if(values[i]>0):
            print(f"Scheduled consumption at Hour {time_str} : {int(values[i]/1000):d} KWH")

else:
    print("Not solved to optimality. Optimization status:", model.status)



In [9]:
example_qa = """
----------
Instruction: Don't charge the car from 8 PM to 10 PM
Answer Code:
```python
upper_bounds[20:22] = [0]*(22-20)
```

----------
Question: Electricity costs are have been increased by 30%, adjust the schedule accordingly
Answer Code:
```python
elec_costs.update((x, y*1.3) for x, y in elec_costs.items())
```
"""

In [10]:
%%capture
agent = ChargingAgent(
    name="Tesla Charging Example",
    source_code=code,
    debug_times=1,
    example_qa="",
    llm_config={
        "request_timeout": 600,
        "seed": 42,
        "config_list": config_list,
    }
)

user = UserProxyAgent(
    "user", max_consecutive_auto_reply=0,
    human_input_mode="NEVER", code_execution_config=False
)

In [11]:
user.initiate_chat(agent, message="Turn charging off between 12 AM and 4 AM")

user (to Tesla Charging Example):

Turn charging off between 12 AM and 4 AM

--------------------------------------------------------------------------------
Tesla Charging Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to Tesla Charging Example):

```python
# upper_bounds adjustment for off time
for i in range(24, 28):
    upper_bounds[i] = 0
```

--------------------------------------------------------------------------------
Tesla Charging Example (to safeguard):


--- Code ---
# upper_bounds adjustment for off time
for i in range(24, 28):
    upper_bounds[i] = 0

--- One-Word Answer: SAFE or DANGER ---


--------------------------------------------------------------------------------
safeguard (to Tesla Charging Example):

SAFE

--------------------------------------------------------------------------------
name 'params_filepath' is not defined
Tesla Charging Example (to writer):



While running the c

KeyboardInterrupt: 

In [10]:
user.initiate_chat(agent, message="Turn charging off between 12 AM and 5 AM")

user (to Tesla Charging Example):

Turn charging off between 12 AM and 5 AM

--------------------------------------------------------------------------------
Tesla Charging Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to Tesla Charging Example):

```python
# Ensure charging does not occur between 12 AM and 5 AM
for hour in range(24,29):
    model.addConstr(x[str(hour)] == 0)
```

--------------------------------------------------------------------------------
Tesla Charging Example (to safeguard):


--- Code ---
# Ensure charging does not occur between 12 AM and 5 AM
for hour in range(24,29):
    model.addConstr(x[str(hour)] == 0)

--- One-Word Answer: SAFE or DANGER ---


--------------------------------------------------------------------------------
safeguard (to Tesla Charging Example):

SAFE

--------------------------------------------------------------------------------
[Errno 2] No such file or di

KeyboardInterrupt: 